In [1]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image
from torchvision.models import resnet50
from sklearn.metrics import roc_auc_score
import models_vit
import cv2
import numpy as np
import torch
import timm
import matplotlib.pylab as plt
import ast

In [2]:
def reshape_transform(tensor, height=14, width=14):
    result = tensor[:, 1:, :].reshape(tensor.size(0),
                                      height, width, tensor.size(2))
    # Bring the channels to the first dimension,
    # like in CNNs.
    result = result.transpose(2, 3).transpose(1, 2)
    return result

In [3]:
def run_grad_cam(model, model_type, target_layers, data_lines):
    model.eval()
    cam = GradCAM(model=model, target_layers=target_layers, reshape_transform=reshape_transform, use_cuda=1)
    for line in data_lines:
        img_path = line.split(' ')[0]
        rgb_img = cv2.imread(data_dir + img_path, 1)[:, :, ::-1]
        rgb_img = cv2.resize(rgb_img, (224, 224))
        rgb_img = np.float32(rgb_img) / 255
        input_tensor = preprocess_image(rgb_img, mean = [0.5056, 0.5056, 0.5056], std = [0.252, 0.252, 0.252])
        grayscale_cam = cam(input_tensor=input_tensor, targets=None)

        grayscale_cam = grayscale_cam[0, :]
        visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=False)
        cv2.imwrite(model_type + img_path, visualization)

In [4]:
@torch.no_grad()
def eval_image(model, target_layers, data_lines, output_file_path):
    outputs = []
    targets = []
    with open(output_file_path, 'a') as output_file:
        for line in lines:
            line_split = line.split()
            img_path = line_split[0]
            imageLabel = line_split[1:num_class + 1]
            imageLabel = [float(i) for i in imageLabel]
            imageLabel = [imageLabel]
            imageLabel = torch.tensor(imageLabel).to(torch.device('cuda'))

            model.eval()
            rgb_img = cv2.imread(data_dir + img_path, 1)[:, :, ::-1]

            rgb_img = cv2.resize(rgb_img, (224, 224))
            rgb_img = np.float32(rgb_img) / 255
            input_tensor = preprocess_image(rgb_img, mean = [0.5056, 0.5056, 0.5056], std = [0.252, 0.252, 0.252])

            output, _ = model(input_tensor.cuda())
            outputs.append(output)
            targets.append(imageLabel)
            acc, res = accuracy(imageLabel, output.sigmoid(), num_class)
        
            output_file.write(f'{img_path}, acc_each_class: {res}, acc_avg: {acc}\n')
        
            
    outputs = torch.cat(outputs, dim = 0).sigmoid().cpu().numpy()
    targets = torch.cat(targets, dim = 0).cpu().numpy()
    
    auc = computeAUROC(targets, outputs, num_class)
    auc_each_class_array = np.array(auc)
    missing_classes_index = np.where(auc_each_class_array == 0)[0]
    # print(missing_classes_index)
    if missing_classes_index.shape[0] > 0:
        print('There are classes that not be predicted during testing,'
              ' the indexes are:', missing_classes_index)

    auc_avg = np.average(auc_each_class_array[auc_each_class_array != 0])
    print(auc_avg)
        

In [5]:
def computeAUROC(dataGT, dataPRED, classCount):
    outAUROC = []
    # print(dataGT.shape, dataPRED.shape)
    for i in range(classCount):
        try:
            outAUROC.append(roc_auc_score(dataGT[:, i], dataPRED[:, i]))
        except:
            outAUROC.append(0.)
    print(outAUROC)
    return outAUROC

In [6]:
def accuracy(dataGT, dataPRED, classCount):
    acc = 0
    pred = (dataPRED >= 0.5).float()
    correct = pred.eq(dataGT[0])
    res = correct.int()
    return torch.sum(correct).item() * 100 / classCount, res.cpu().numpy()

In [2]:
data_dir = "/path/to/dataset"
img_dir = "./data_splits/chestxray/test_official.txt"

out_dir = '/path/to/output_dir'

base_model = out_dir + 'base_model/'
low_rank = out_dir + 'low_rank_model/'

base_auc_out = base_model + 'base.txt'
low_auc_out = low_rank + 'low.txt'

num_class = 14

with open(img_dir, 'r') as input_file:
    lines = input_file.readlines()

In [8]:
# GRAD CAM and EVAL BASE MODEL
model = models_vit.__dict__['vit_base_patch16'](img_size=224,
                                            num_classes=14,
                                            drop_rate=0,
                                            drop_path_rate=0,
                                            global_pool=True)
checkpoint = torch.load('/path/to/base_model_checkpoint')
model.load_state_dict(checkpoint['model'], strict=True)
model.to(torch.device('cuda'))
target_layers = [model.blocks[-1].norm1]

run_grad_cam(model, base_model, target_layers, lines)
eval_image(model, target_layers, lines, base_auc_out)

[0.7827995028467994, 0.9053250454635379, 0.8444961162607254, 0.7041887629842073, 0.8292594733272587, 0.7491918601484793, 0.7383613396552178, 0.8690073974268019, 0.7637604296227873, 0.8653487024375923, 0.9312777168387507, 0.841948746623469, 0.8129848719018493, 0.9090201744869773]
0.824783581430318


In [9]:
# GRAD CAM and EVAL LOW RANK MODEL
model = models_vit.__dict__['vit_base_patch16'](img_size=224,
                                            num_classes=14,
                                            drop_rate=0,
                                            drop_path_rate=0,
                                            global_pool=True)
checkpoint = torch.load('/path/to/low_rank_model_checkpoint')
model.load_state_dict(checkpoint['model'], strict=True)
target_layers = [model.blocks[-1].norm1]

run_grad_cam(model, low_rank, target_layers, lines)
eval_image(model, target_layers, lines, low_auc_out)

[0.7838100123285259, 0.9046906669700557, 0.8452978043376627, 0.7092338434355051, 0.8298423866025637, 0.7548728893223196, 0.739892450255455, 0.8692214746859062, 0.7671564203337086, 0.8653401794644549, 0.9310669018663773, 0.8479242871311056, 0.8118228233575656, 0.921626266033384]
0.8272713147231849


In [10]:
# Define the file paths
file1_path = base_auc_out # Replace with the path to the first file
file2_path = low_auc_out  # Replace with the path to the second file
output_file_path = out_dir + 'result.txt'  # Replace with the desired output file path

# Initialize dictionaries to store the 'C' values from both files
file1_c_values = {}
file2_c_values = {}

# Initialize a list to store the results
results = []

# Read and process data from the first file
with open(file1_path, 'r') as file1:
    for line in file1:
        parts = line.strip().split(',')
        if len(parts) == 3:
            a, b, c = parts
            file1_c_values[a] = float(c.split(':')[-1])

# Read and process data from the second file
with open(file2_path, 'r') as file2:
    for line in file2:
        parts = line.strip().split(',')
        if len(parts) == 3:
            a, b, c = parts
            file2_c_value = float(c.split(':')[-1])
            if a in file1_c_values and file2_c_value > file1_c_values[a]:
                result_line = f"image_path: {a}, base_acc: {file1_c_values[a]}, low_acc: {file2_c_value}"
                results.append(result_line)

# Write the results to the output file
with open(output_file_path, 'w') as output_file:
    for result_line in results:
        output_file.write(result_line + '\n')

print(f"Results saved to {output_file_path}.")


Results saved to ...


In [8]:
import re
# Step 1: Read data from all four files into dictionaries
file1_data = []
file2_data = {}
file3_data = {}
file4_data = {}

# Read data from the first file (name, a:A, b:B)
with open(out_dir + 'result.txt', 'r') as file1:
    for line in file1:
        parts = line.strip().split(', ')
        name = parts[0].split(': ')[1]
        file1_data.append(name)

# Read data from the second file (name, c:C)
with open(base_auc_out, 'r') as file2:
    for line in file2:
        parts = line.strip().split(', ')
        name = parts[0]
        preds = parts[1].split(':')[1]
        match = re.search(r'\[\[(.*?)\]\]', preds)
        if match:
            numeric_data = match.group(1).strip().split()
            numeric_array = [int(val) for val in numeric_data]

        file2_data[name] = numeric_array
        

# Read data from the third file (name, d:D)
with open(low_auc_out, 'r') as file3:
    for line in file3:
        parts = line.strip().split(', ')
        name = parts[0]
        preds = parts[1].split(':')[1]
        match = re.search(r'\[\[(.*?)\]\]', preds)
        if match:
            numeric_data = match.group(1).strip().split()
            numeric_array = [int(val) for val in numeric_data]

        file3_data[name] = numeric_array

# Read data from the fourth file (name data)
with open('./data_splits/chestxray/test_official.txt', 'r') as file4:
    for line in file4:
        lineItems = line.split()
        imagePath = lineItems[0]
        imageLabel = lineItems[1:15]
        imageLabel = [int(i) for i in imageLabel]
        file4_data[imagePath] = imageLabel

def get_preds(acc_each_class, gt):
    preds = []
    for i in range(len(gt)):
        if acc_each_class[i] == 1:
            preds.append(gt[i])
        else:
            preds.append(1) if gt[i] == 0 else preds.append(0)
    return preds
    
    
# Step 2-4: Process the data and create the new format
output_data = []
for name in file1_data:
    if name in file2_data and name in file3_data and name in file4_data:
        c1 = file2_data[name]
        c2 = file3_data[name]
        data = file4_data[name]
        preds_c1 = get_preds(c1, data)
        preds_c2 = get_preds(c2, data)
        new_line = f"{name}, base: {preds_c1}, low_rank: {preds_c2}, GT: {data}"
        output_data.append(new_line)

# Step 5: Write the new formatted data to a fifth file
with open(out_dir + 'new_results.txt', 'w') as file5:
    file5.writelines('\n'.join(output_data))
